In [1]:
import json
import os
from pprint import pprint
from tqdm import tqdm
from pathlib import Path
from utils import load_dataset

DATA_PATH = Path('data/json')
# files = os.listdir(DATA_PATH)
data_paths = list(DATA_PATH.glob('*.json'))
print(f'Loading {len(data_paths)} files...')
for path in data_paths:
    print('\t' + path.name)

data_by_journal = {path.stem: load_dataset(path) for path in data_paths}
print(f"data_by_journal has keys {data_by_journal.keys()}")

all_data = [record for data in data_by_journal.values() for record in data]
print(f'Loaded {len(all_data)} records')

Loading 8 files...
	Earth_Science_Reviews.json
	Earth_Science_Research.json
	Planetary_Research.json
	Planetary_Reviews.json
	salvaged_articles.json
	Astro_Reviews.json
	Astro_Research.json
	doi_articles.json
data/json/Earth_Science_Reviews.json: 994/1000 have all required keys
data/json/Earth_Science_Research.json: 1000/1000 have all required keys
data/json/Planetary_Research.json: 964/1000 have all required keys
data/json/Planetary_Reviews.json: 994/1000 have all required keys
data/json/salvaged_articles.json: 50021/72374 have all required keys
data/json/Astro_Reviews.json: 996/1000 have all required keys
data/json/Astro_Research.json: 981/1000 have all required keys
data/json/doi_articles.json: 1898/1898 have all required keys
data_by_journal has keys dict_keys(['Earth_Science_Reviews', 'Earth_Science_Research', 'Planetary_Research', 'Planetary_Reviews', 'salvaged_articles', 'Astro_Reviews', 'Astro_Research', 'doi_articles'])
Loaded 57848 records


In [2]:
import re

# Define bibcode and citation patterns
lastname = r"[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*(?:'[A-Z][a-zA-ZÀ-ÖØ-öø-ÿ-]*)?"
year = r"\(?(\d{4}[a-z]?)\)?"
name_sep = r",?\s"
INLINE_CITATION_PATTERN = fr"({lastname}(?:{name_sep}{lastname})*(?: et al.?)?)\s*{year}"

# Compile the regex pattern
INLINE_REGEX = re.compile(INLINE_CITATION_PATTERN)

In [3]:

def bibcode_regex(author: str, year: str):
    """
    Given first author and year, return a regex pattern for the
    corresponding bibcode
    """
    initial = author[0]
    year = year[:4]  # cut off any letters at the end
    pattern = fr'^{year}.*{initial}$'
    return re.compile(pattern)


def citation_bibcode_matches(inline_citation: tuple[str, str], references: list[str]) -> int:
    """
    Given an inline citation and a list of references, return the number of
    references that match the inline citation's bibcode regex pattern
    """
    pattern = bibcode_regex(*inline_citation)
    return [s for s in references if pattern.match(s)]



In [24]:
citation = ('Charlot', '1996')
bibcodes = citation_bibcode_matches(citation, paper['reference'])
print(f"Bibcodes: {bibcodes}")
matching_dois = {doi for bibcode in bibcodes if (
    doi := author_and_bibcode_to_doi(citation[0], bibcode, all_data))}

Bibcodes: ['1996A&AS..115..339C', '1996ASPC...98..275C', '1996ApJ...457..625C']
No matching records for bibcode 1996A&AS..115..339C
No matching records for bibcode 1996ASPC...98..275C
No matching records for bibcode 1996ApJ...457..625C


In [25]:
for record in all_data:
    if record['bibcode'] =='1996ApJ...457..625C':
        print(record['doi'])

In [43]:
import pysbd
import unicodedata

def get_inline_citations(text: str) -> list[tuple[str, str]]:
    return [match.groups() for match in INLINE_REGEX.finditer(text)]

def strip_accents(text):
    """
    Strip accents from input string. Helpful for normalizing author names
    e.g. Schönberger -> Schonberger
    """
    # Normal Form Decomposition: separates base characters from their diacritics
    text = unicodedata.normalize('NFD', text)

    # Skips any chars that are accent marks
    return ''.join(c for c in text if unicodedata.category(c) != 'Mn')

def first_author_normed(authors: list[str]) -> str:
    """
    Given a list of authors, return the first author
    e.g. ['Schönrich, ralph', 'binney, james'] -> 'schonrich'
    """
    return strip_accents(authors[0].split(',')[0]).lower()

def author_and_bibcode_to_doi(author, bibcode, all_records):
    """
    Given an author and bibcode, return the doi if there is a unique match
    """
    matching_records = [record for record in all_records if record['bibcode'] == bibcode]

    if not matching_records:
        # print(f"No matching records for bibcode {bibcode}")
        return None

    # TODO: currently assumes that if more than one record matches the bibcode then
    # they are duplicates. This may not be accurate! Should be checked or a deduplication
    # step should be added.
    
    record = matching_records[0]
    author_normed = strip_accents(author).lower()
    record_author = first_author_normed(record['author'])
    
    # Check if normalized author name is in the list of authors on this record
    if author_normed != record_author:
        return None

    # Citation author name matches the record author name, so it's a match! 
    return record['doi'][0]


print(author_and_bibcode_to_doi('Schönrich', '2009MNRAS.396..203S', all_data))


10.1111/j.1365-2966.2009.14750.x


In [44]:
def sentence_to_example(record, sentence, all_records):
    """
    """
    def citation_to_doi(citation):
        """
        Takes a single inline citation as tuple of (author, year) and determines if there is a unique
        matching bibcode in the record's references. If so, it continues to look for a unique
        doi matching that bibcode in the entire dataset. 
        
        If there are multiple matching bibcodes, it attempts to resolve the doi by checking each matched
        bibcode's record for matching author.

        It returns the doi if resolved, otherwise None.
        """
        bibcodes = citation_bibcode_matches(citation, record['reference'])
        if len(bibcodes) > 1:
            # If there are multiple matches, attempt to resolve by author
            dois = {doi for bibcode in bibcodes if (doi := author_and_bibcode_to_doi(citation[0], bibcode, all_records))}
            # If only one matching doi, return it
            if len(dois) == 1:
                return dois.pop()
            else:
                # print(f"doi's found for {citation}: {dois}")
                return None
            
        if len(bibcodes) != 1:
            return None

        # Take the bibcode and look for a unique corresponding doi
        matching_dois = [record['doi'][0]
                         for record in all_records if record['bibcode'] == bibcodes[0]]
        if len(matching_dois) != 1:
            return None
        return matching_dois[0]

    inline_citations = get_inline_citations(sentence)
    citation_dois = []
    for citation in inline_citations:
        if not (doi := citation_to_doi(citation)):
            break
        citation_dois.append(doi)

    # If all citations resolved to dois, return the example
    # TODO: is this too strict?
    if len(inline_citations) != len(citation_dois):
        return None
    return {
        'source_doi': record['doi'][0],
        'sentence': sentence,
        'citation_dois': citation_dois
    }


def create_examples_from_record(record, all_records):
    splitter = pysbd.Segmenter(language="en", clean=False)
    sentences = [s for s in splitter.segment(record['body']) if len(s) > 40]
    return [
        example for sentence in sentences if (example := sentence_to_example(record, sentence, all_records))
    ]

In [45]:
paper = data_by_journal['Astro_Research'][0]
examples = create_examples_from_record(paper, all_data)

In [30]:
# nontrivial_examples = [ex for ex in examples if len(ex['citation_dois']) > 0]
# print(f"Found {len(nontrivial_examples)} non-trivial examples")

Found 57 non-trivial examples


In [46]:
from itertools import chain

all_review_papers = list(chain(*[data_by_journal[journal] for journal in ['Astro_Reviews', 'Earth_Science_Reviews', 'Planetary_Reviews']]))
print(len(all_review_papers))

for paper in tqdm(all_review_papers):
    examples = create_examples_from_record(paper, all_data)
    trivial_examples, nontrivial_examples = [], []
    for example in examples:
        if len(example['citation_dois']) > 0:
            nontrivial_examples.append(example)
        else:
            trivial_examples.append(example)

    with open('data/examples_trivial.json', 'a') as f:
        for example in trivial_examples:
            f.write(json.dumps(example) + '\n')
    with open('data/examples_nontrivial.json', 'a') as f:
        for example in nontrivial_examples:
            f.write(json.dumps(example) + '\n')

2984


 32%|███▏      | 960/2984 [4:19:42<9:07:33, 16.23s/it]   


KeyboardInterrupt: 

None


In [9]:
for journal in data_by_journal:
    data = data_by_journal[journal]
    for i, record in enumerate(data):
        if record['bibcode'] == '2009MNRAS.396..203S':
            print(f"Found in {journal}")
            print(record['author'])
            break
            

Found in Astro_Research
['Schönrich, Ralph', 'Binney, James']
Found in doi_articles
['Schönrich, Ralph', 'Binney, James']
